In [1]:
# enable autoreload
%load_ext autoreload
%autoreload 2

In [2]:
from askharrison.arxiv_search import expand_arxiv_query, run_multi_arixv_queries

In [3]:
problem_statement = """RAG(retrieval augmented generation) sometimes the sources retrieved are relevant but not enough to answer the question user ask. How do research approach this case"""

In [4]:
search_queries = expand_arxiv_query(problem_statement)

In [5]:
search_queries

['Relevant source retrieval in RAG',
 'Dealing with inadequate sources in RAG',
 'Improving source retrieval in Retrieval Augmented Generation',
 'Strategies to enhance relevancy of retrieved sources in RAG',
 'Techniques for better source retrieval in RAG',
 'Addressing relevancy in source retrieval in RAG',
 'Approaches to improve source usefulness in RAG',
 'Optimizing source retrieval in Retrieval Augmented Generation',
 'Methods to boost relevant source retrieval in Retrieval Augmented Generation',
 'Resolving relevance issue in source retrieval in RAG']

In [6]:
arxiv_query_results = run_multi_arixv_queries(search_queries)

100%|██████████| 10/10 [00:24<00:00,  2.45s/it]


In [7]:
len(arxiv_query_results.items())

10

In [8]:
# flatten arxiv_query_results
all_results = []
for query in arxiv_query_results:
    for result in arxiv_query_results[query]:
        all_results.append(result)

# make arxiv query results a dataframe and create a new dataframe with only unique entry_id
import pandas as pd

arixv_result_df = pd.DataFrame(all_results)
unique_arixv_result_df = arixv_result_df.drop_duplicates(subset='entry_id')

In [9]:
arixv_result_df.shape, unique_arixv_result_df.shape

((200, 13), (95, 13))

In [10]:
arixv_result_df.groupby('entry_id').count().sort_values('title', ascending=False).head(10)

,authors,categories,comment,doi,journal_ref,links,pdf_url,primary_category,published,summary,title,updated
entry_id,,,,,,,,,,,,
http://arxiv.org/abs/2410.07176v1,7,7,7,0,0,7,7,7,7,7,7,7
http://arxiv.org/abs/2405.19207v1,7,7,7,0,0,7,7,7,7,7,7,7
http://arxiv.org/abs/2402.17497v1,7,7,0,0,0,7,7,7,7,7,7,7
http://arxiv.org/abs/2402.11891v1,6,6,0,0,0,6,6,6,6,6,6,6
http://arxiv.org/abs/2410.03780v1,6,6,0,0,0,6,6,6,6,6,6,6
http://arxiv.org/abs/2404.07221v2,6,6,0,0,0,6,6,6,6,6,6,6
http://arxiv.org/abs/2406.13213v2,6,6,6,0,0,6,6,6,6,6,6,6
http://arxiv.org/abs/2410.01782v1,5,5,5,0,0,5,5,5,5,5,5,5
http://arxiv.org/abs/2210.02627v1,5,5,5,0,0,5,5,5,5,5,5,5


In [11]:
from askharrison.prompts.content_curation import create_arxiv_filtering_prompt

In [12]:
help(create_arxiv_filtering_prompt)

Help on function create_arxiv_filtering_prompt in module askharrison.prompts.content_curation:

create_arxiv_filtering_prompt(problem_statement: str, doc_abstract: str)



In [13]:
# create a prompt for each arxiv entry
arxiv_reranking_prompts = [create_arxiv_filtering_prompt(problem_statement, 
                                         record['title']+"\n"+record['summary']) for record in unique_arixv_result_df.to_dict(orient='records')]

In [14]:
from askharrison.llm_models import parallel_llm_processor, process_question, safe_eval, extract_python_code

In [15]:
reranking_llm_response = parallel_llm_processor(arxiv_reranking_prompts, llm_function=process_question, 
                                                max_workers=8)

Processing prompts: 100%|██████████| 95/95 [00:30<00:00,  3.11it/s]


In [23]:
llm_responses_results = [safe_eval(extract_python_code(response)) for response in reranking_llm_response]
# filter out empty responses

unique_arixv_result_df.shape, len(llm_responses_results)

((95, 13), 95)

In [25]:
# extract reasoning, is_direct, 'is_relevant' from llm_responses_results if it is not empty, and add to unique_arixv_result_df
unique_arixv_result_df['reasoning'] = [response['reasoning'] if response else None for response in llm_responses_results]
unique_arixv_result_df['is_direct'] = [response['is_direct'] if response else None for response in llm_responses_results]
unique_arixv_result_df['is_relevant'] = [response['is_relevant'] if response else None for response in llm_responses_results]


C:\Users\alist\AppData\Local\Temp\ipykernel_93080\658567496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_arixv_result_df['reasoning'] = [response['reasoning'] if response else None for response in llm_responses_results]
C:\Users\alist\AppData\Local\Temp\ipykernel_93080\658567496.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_arixv_result_df['is_direct'] = [response['is_direct'] if response else None for response in llm_responses_results]
C:\Users\alist\AppData\Local\Temp\ipykernel_

In [31]:
# increase max column width in pandas
pd.set_option('display.max_colwidth', None)

In [32]:
unique_arixv_result_df.query('is_direct == True')\
    [['title','entry_id', 'summary', 'is_relevant']]

,title,entry_id,summary,is_relevant
1,RAGBench: Explainable Benchmark for Retrieval-Augmented Generation Systems,http://arxiv.org/abs/2407.11005v1,"Retrieval-Augmented Generation (RAG) has become a standard architectural\npattern for incorporating domain-specific knowledge into user-facing chat\napplications powered by Large Language Models (LLMs). RAG systems are\ncharacterized by (1) a document retriever that queries a domain-specific corpus\nfor context information relevant to an input query, and (2) an LLM that\ngenerates a response based on the provided query and context. However,\ncomprehensive evaluation of RAG systems remains a challenge due to the lack of\nunified evaluation criteria and annotated datasets. In response, we introduce\nRAGBench: the first comprehensive, large-scale RAG benchmark dataset of 100k\nexamples. It covers five unique industry-specific domains and various RAG task\ntypes. RAGBench examples are sourced from industry corpora such as user\nmanuals, making it particularly relevant for industry applications. Further, we\nformalize the TRACe evaluation framework: a set of explainable and actionable\nRAG evaluation metrics applicable across all RAG domains. We release the\nlabeled dataset at https://huggingface.co/datasets/rungalileo/ragbench.\nRAGBench explainable labels facilitate holistic evaluation of RAG systems,\nenabling actionable feedback for continuous improvement of production\napplications. Thorough extensive benchmarking, we find that LLM-based RAG\nevaluation methods struggle to compete with a finetuned RoBERTa model on the\nRAG evaluation task. We identify areas where existing approaches fall short and\npropose the adoption of RAGBench with TRACe towards advancing the state of RAG\nevaluation systems.",5.0
3,Open-RAG: Enhanced Retrieval-Augmented Reasoning with Open-Source Large Language Models,http://arxiv.org/abs/2410.01782v1,"Retrieval-Augmented Generation (RAG) has been shown to enhance the factual\naccuracy of Large Language Models (LLMs), but existing methods often suffer\nfrom limited reasoning capabilities in effectively using the retrieved\nevidence, particularly when using open-source LLMs. To mitigate this gap, we\nintroduce a novel framework, Open-RAG, designed to enhance reasoning\ncapabilities in RAG with open-source LLMs. Our framework transforms an\narbitrary dense LLM into a parameter-efficient sparse mixture of experts (MoE)\nmodel capable of handling complex reasoning tasks, including both single- and\nmulti-hop queries. Open-RAG uniquely trains the model to navigate challenging\ndistractors that appear relevant but are misleading. As a result, Open-RAG\nleverages latent learning, dynamically selecting relevant experts and\nintegrating external knowledge effectively for more accurate and contextually\nrelevant responses. In addition, we propose a hybrid adaptive retrieval method\nto determine retrieval necessity and balance the trade-off between performance\ngain and inference speed. Experimental results show that the Llama2-7B-based\nOpen-RAG outperforms state-of-the-art LLMs and RAG models such as ChatGPT,\nSelf-RAG, and Command R+ in various knowledge-intensive tasks. We open-source\nour code and models at https://openragmoe.github.io/",5.0
6,A Multi-Source Retrieval Question Answering Framework Based on RAG,http://arxiv.org/abs/2405.19207v1,"With the rapid development of large-scale language models,\nRetrieval-Augmented Generation (RAG) has been widely adopted. However, existing\nRAG paradigms are inevitably influenced by erroneous retrieval information,\nthereby reducing the reliability and correctness of generated results.\nTherefore, to improve the relevance of retrieval information, this study\nproposes a method that replaces traditional retrievers with GPT-3.5, leveraging\nits vast corpus knowledge to generate retrieval information. We also propose a\nweb retrieval based method to implement fine-grained knowledge retrieval,\nUtilizing the powerful reasoning c